In [34]:
import pandas as pd
import numpy as np
import duckdb
import os
import glob
from memory_limits import duck_options
from datetime import datetime
from pathlib import Path


In [36]:
data_dir = Path("../data")

mlhd_path = data_dir / "solo-artist-count/chunk.parquet"
musicbrainz_path = data_dir / "musicbrainz.db"
mlhd_ids_path = data_dir / "mlhd_ids.parquet"


conn = duckdb.connect(config=duck_options())

-1 -1 270210306048


In [37]:
conn.execute(f"""create temp table mlhd_ids as 
                select * from read_parquet('{mlhd_ids_path}') 
                ORDER BY entity_uuid""")

In [38]:
conn.execute(f"attach '{musicbrainz_path}' as musicbrainz")

In [39]:
# map musicbrainz to mlhd_ids
conn.execute(f"""
    create temp table artists as
    SELECT 
        artist_uuid.id_num AS artist_id,
        mb_artist.gender AS gender,
        mb_artist.type AS type
    FROM musicbrainz.mb_artist as mb_artist
    RIGHT JOIN (SELECT * FROM mlhd_ids WHERE entity_type = 'artist_ids') 
                 AS artist_uuid
        ON artist_uuid.entity_uuid = mb_artist.artist_id
        """)

In [40]:
conn.execute("detach musicbrainz")

In [41]:
# read training data
conn.execute(f"""
        create or replace table mlhd_train as 
        with sorted_data as(
        select *, percent_rank() over (order by last_time) as percentile
        from read_parquet('{mlhd_path}'))
        
        select user_id, artist_id, count,
            case
                when percentile >= 0.8 then 'test'
                else 'train'
            end as label
        from sorted_data """)



FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [42]:
conn.execute("""create or replace table mlhd_train
             as select user_id, artist_id, count,
             from mlhd_train
             where label='train'
             """)

In [43]:
# preprocessing on train


# exclude artists with fewer than 15 total listens across all users
del_artist_unpop = conn.execute(""" 
    DELETE FROM mlhd_train
    WHERE artist_id NOT in (
        SELECT artist_id
        FROM mlhd_train
        GROUP BY artist_id
        HAVING SUM(count) >= 15
    );
""")
deleted_rows = del_artist_unpop.fetchall()[0][0]
print(f"Number rows deleted for artists with <15 total listens: {deleted_rows}")

# exclude users with fewer than 15 distinct artists in their data
del_user_artist = conn.execute("""
    DELETE FROM mlhd_train
    WHERE user_id NOT IN (
        SELECT user_id
        FROM mlhd_train
        GROUP BY user_id
        HAVING COUNT(DISTINCT artist_id) >= 15
    );
""")

deleted_rows = del_user_artist.fetchall()[0][0]
print(f"Number of rows deleted for users with <15 artists : {deleted_rows}")

Number rows deleted for artists with <15 total listens: 41111


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Number of rows deleted for users with <15 artists : 202051


In [44]:
conn.execute("select * from mlhd_train").fetchdf()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,user_id,artist_id,count
0,100474387,200106378,1
1,100083599,200233107,1
2,100102371,200286912,1
3,100141357,200123173,1
4,100325357,200286890,1
...,...,...,...
161377806,100167238,200248040,1
161377807,100534006,200010460,62
161377808,100234586,200017705,140
161377809,100327002,200390334,1
